In [2]:
import pandas as pd
import numpy as np
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
from keras.utils import to_categorical
from keras.utils import img_to_array, load_img
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import math
import datetime
import time

import torch
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [11]:
# Default dimensions we found online
img_width, img_height = 299, 299

# Create a bottleneck file
top_model_weights_path = 'pytorch_bottleneck_fc_model_inceptionv3.h5'

# Loading up our datasets
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test'

# Number of epochs to train top model
epochs = 100  # This has been changed after multiple model runs
# Batch size used by flow_from_directory and predict_generator
batch_size = 50

In [13]:
# Load pre-trained InceptionV3 model in PyTorch
pytorch_model = models.inception_v3(pretrained=True)

# Modify the last layer for feature extraction
num_ftrs = pytorch_model.fc.in_features
pytorch_model.fc = nn.Identity()

# Define transforms for images
transform = transforms.Compose([
    transforms.Resize((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = ImageFolder(train_data_dir, transform=transform)
validation_dataset = ImageFolder(validation_data_dir, transform=transform)
test_dataset = ImageFolder(test_data_dir, transform=transform)

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


#1
# # Feature extraction for training data
# start = datetime.datetime.now()
# train_features = []
# train_labels = []

# with torch.no_grad():
#     for images, labels in train_loader:
#         features = pytorch_model(images)
#         train_features.append(features)
#         train_labels.append(labels)

# train_features = torch.cat(train_features)
# train_labels = torch.cat(train_labels)
# np.save('train_features_inceptionv3.npy', train_features.numpy())
# np.save('train_labels_inceptionv3.npy', train_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for training data:', elapsed)

# # Feature extraction for validation data
# start = datetime.datetime.now()
# validation_features = []
# validation_labels = []

# with torch.no_grad():
#     for images, labels in validation_loader:
#         features = pytorch_model(images)
#         validation_features.append(features)
#         validation_labels.append(labels)

# validation_features = torch.cat(validation_features)
# validation_labels = torch.cat(validation_labels)
# np.save('validation_features_inceptionv3.npy', validation_features.numpy())
# np.save('validation_labels_inceptionv3.npy', validation_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for validation data:', elapsed)

# # Feature extraction for test data
# start = datetime.datetime.now()
# test_features = []
# test_labels = []

# with torch.no_grad():
#     for images, labels in test_loader:
#         features = pytorch_model(images)
#         test_features.append(features)
#         test_labels.append(labels)

# test_features = torch.cat(test_features)
# test_labels = torch.cat(test_labels)
# np.save('test_features_inceptionv3.npy', test_features.numpy())
# np.save('test_labels_inceptionv3.npy', test_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for test data:', elapsed)

#2
# # Feature extraction for training data
# start = datetime.datetime.now()
# train_features = []
# train_labels = []

# with torch.no_grad():
#     for images, labels in train_loader:
#         features = pytorch_model(images)
#         train_features.append(features)
#         train_labels.append(labels)

# # Check if train_features contain InceptionOutputs, if so, convert them to tensors
# if isinstance(train_features[0], torch.nn.modules.container.Sequential):
#     train_features = [features[0] for features in train_features]

# train_features = torch.cat(train_features)
# train_labels = torch.cat(train_labels)
# np.save('train_features_inceptionv3.npy', train_features.numpy())
# np.save('train_labels_inceptionv3.npy', train_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for training data:', elapsed)

# # Feature extraction for validation data
# start = datetime.datetime.now()
# validation_features = []
# validation_labels = []

# with torch.no_grad():
#     for images, labels in validation_loader:
#         features = pytorch_model(images)
#         validation_features.append(features)
#         validation_labels.append(labels)

# # Check if validation_features contain InceptionOutputs, if so, convert them to tensors
# if isinstance(validation_features[0], torch.nn.modules.container.Sequential):
#     validation_features = [features[0] for features in validation_features]

# validation_features = torch.cat(validation_features)
# validation_labels = torch.cat(validation_labels)
# np.save('validation_features_inceptionv3.npy', validation_features.numpy())
# np.save('validation_labels_inceptionv3.npy', validation_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for validation data:', elapsed)

# # Feature extraction for test data
# start = datetime.datetime.now()
# test_features = []
# test_labels = []

# with torch.no_grad():
#     for images, labels in test_loader:
#         features = pytorch_model(images)
#         test_features.append(features)
#         test_labels.append(labels)

# # Check if test_features contain InceptionOutputs, if so, convert them to tensors
# if isinstance(test_features[0], torch.nn.modules.container.Sequential):
#     test_features = [features[0] for features in test_features]

# test_features = torch.cat(test_features)
# test_labels = torch.cat(test_labels)
# np.save('test_features_inceptionv3.npy', test_features.numpy())
# np.save('test_labels_inceptionv3.npy', test_labels.numpy())
# end = datetime.datetime.now()
# elapsed = end - start
# print('Feature extraction time for test data:', elapsed)

#3
# Feature extraction for training data
start = datetime.datetime.now()
train_features = []
train_labels = []

with torch.no_grad():
    for images, labels in train_loader:
        features = pytorch_model(images)

        # Extract the desired features from InceptionOutputs
        features = features.logits

        train_features.append(features)
        train_labels.append(labels)

train_features = torch.cat(train_features)
train_labels = torch.cat(train_labels)
np.save('train_features_inceptionv3.npy', train_features.numpy())
np.save('train_labels_inceptionv3.npy', train_labels.numpy())
end = datetime.datetime.now()
elapsed = end - start
print('Feature extraction time for training data:', elapsed)

# Feature extraction for validation data
start = datetime.datetime.now()
validation_features = []
validation_labels = []

with torch.no_grad():
    for images, labels in validation_loader:
        features = pytorch_model(images)

        # Extract the desired features from InceptionOutputs
        features = features.logits

        validation_features.append(features)
        validation_labels.append(labels)

validation_features = torch.cat(validation_features)
validation_labels = torch.cat(validation_labels)
np.save('validation_features_inceptionv3.npy', validation_features.numpy())
np.save('validation_labels_inceptionv3.npy', validation_labels.numpy())
end = datetime.datetime.now()
elapsed = end - start
print('Feature extraction time for validation data:', elapsed)

# Feature extraction for test data
start = datetime.datetime.now()
test_features = []
test_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        features = pytorch_model(images)

        # Extract the desired features from InceptionOutputs
        features = features.logits

        test_features.append(features)
        test_labels.append(labels)

test_features = torch.cat(test_features)
test_labels = torch.cat(test_labels)
np.save('test_features_inceptionv3.npy', test_features.numpy())
np.save('test_labels_inceptionv3.npy', test_labels.numpy())
end = datetime.datetime.now()
elapsed = end - start
print('Feature extraction time for test data:', elapsed)


# Load features and labels
train_data = np.load('train_features_inceptionv3.npy')
train_labels = np.load('train_labels_inceptionv3.npy')
validation_data = np.load('validation_features_inceptionv3.npy')
validation_labels = np.load('validation_labels_inceptionv3.npy')
test_data = np.load('test_features_inceptionv3.npy')
test_labels = np.load('test_labels_inceptionv3.npy')

# Convert target labels to one-hot encoded format
train_labels = to_categorical(train_labels, num_classes=20)
validation_labels = to_categorical(validation_labels, num_classes=20)
test_labels = to_categorical(test_labels, num_classes=20)

Feature extraction time for training data: 0:05:29.645817
Feature extraction time for validation data: 0:00:35.083273
Feature extraction time for test data: 0:00:38.137717


In [14]:
# Define the model architecture
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(512, activation=keras.layers.LeakyReLU(
    alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(128, activation=keras.layers.LeakyReLU(
    alpha=0.3)))
model.add(Dropout(0.3))
model.add(Dense(units=20, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc'])

early_stopping = EarlyStopping(
    monitor='val_acc', patience=20, verbose=1, mode='max')

# Train the model
history = model.fit(train_data, train_labels,
                    epochs=200,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels),
                    callbacks=[early_stopping])

# Save the trained model
model.save_weights(top_model_weights_path)

# Evaluate the model on the test data
(eval_loss, eval_accuracy) = model.evaluate(
    test_data, test_labels, batch_size=batch_size, verbose=1)

# Print the evaluation results
print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))

Epoch 1/200
31/31 [==============================] - 1s 20ms/step - loss: 3.2701 - acc: 0.0554 - val_loss: 3.0177 - val_acc: 0.0732
Epoch 2/200
31/31 [==============================] - 0s 13ms/step - loss: 3.0924 - acc: 0.0646 - val_loss: 2.9581 - val_acc: 0.0915
Epoch 3/200
31/31 [==============================] - 0s 15ms/step - loss: 2.9970 - acc: 0.0943 - val_loss: 2.9232 - val_acc: 0.1037
Epoch 4/200
31/31 [==============================] - 0s 15ms/step - loss: 2.9187 - acc: 0.1088 - val_loss: 2.8883 - val_acc: 0.1159
Epoch 5/200
31/31 [==============================] - 0s 14ms/step - loss: 2.8571 - acc: 0.1292 - val_loss: 2.8523 - val_acc: 0.1524
Epoch 6/200
31/31 [==============================] - 0s 13ms/step - loss: 2.7942 - acc: 0.1549 - val_loss: 2.8132 - val_acc: 0.1646
Epoch 7/200
31/31 [==============================] - 0s 14ms/step - loss: 2.6841 - acc: 0.1819 - val_loss: 2.7596 - val_acc: 0.1768
Epoch 8/200
31/31 [==============================] - 1s 17ms/step - loss: 2.

In [ ]:
# Model summary
model.summary()

In [ ]:
# Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
model.evaluate(test_data, test_labels)

In [ ]:
print('test data', test_data)
preds = np.round(model.predict(test_data), 0)
# to fit them into classification metrics and confusion metrics, some additional modificaitions are required
print('rounded test_labels', preds)

In [ ]:
animals = ['bear', 'cougar', 'coyote', 'cow', 'crocodiles', 'deer', 'elephant', 'giraffe', 'goat',
           'gorilla', 'horse', 'kangaroo', 'leopard', 'lion', 'panda', 'penguin', 'sheep', 'skunk', 'tiger', 'zebra']
classification_metrics = metrics.classification_report(
    test_labels, preds, target_names=animals)
print(classification_metrics)

In [ ]:
# Since our data is in dummy format we put the numpy array into a dataframe and call idxmax axis=1 to return the column
# label of the maximum value thus creating a categorical variable
# Basically, flipping a dummy variable back to it's categorical variable
categorical_test_labels = pd.DataFrame(test_labels).idxmax(axis=1)
categorical_preds = pd.DataFrame(preds).idxmax(axis=1)

In [ ]:
confusion_matrix = confusion_matrix(categorical_test_labels, categorical_preds)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues,
                          figsize=(10, 8)):  # Adjust the figsize as per your preference
    # Add Normalization Option
    '''prints pretty confusion metric with normalization option '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # Set figure size
    plt.figure(figsize=figsize)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))

    # Rotate x-labels by 90 degrees
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        # Increase x-coordinate for more horizontal space
        plt.text(j, i, format(cm[i, j], fmt), ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(confusion_matrix, ['bear', 'cougar', 'coyote', 'cow', 'crocodiles', 'deer', 'elephant', 'giraffe',
                      'goat', 'gorilla', 'horse', 'kangaroo', 'leopard', 'lion', 'panda', 'penguin', 'sheep', 'skunk', 'tiger', 'zebra'])

In [ ]:
# Those numbers are all over the place. Now turning normalize= True
plot_confusion_matrix(confusion_matrix,
                      ['bear', 'cougar', 'coyote', 'cow', 'crocodiles', 'deer', 'elephant', 'giraffe', 'goat', 'gorilla',
                          'horse', 'kangaroo', 'leopard', 'lion', 'panda', 'penguin', 'sheep', 'skunk', 'tiger', 'zebra'],
                      normalize=True)

In [ ]:
import time
import torch
import torchvision.transforms as transforms
from PIL import Image
from keras.utils import img_to_array, load_img


def read_image(path):
    image = load_img(path, target_size=(299, 299))
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = image.reshape((1,) + image.shape)  # Add batch dimension
    return image


def test_single_image(path):
    animals = ['bear', 'cougar', 'coyote', 'cow', 'crocodiles', 'deer', 'elephant', 'giraffe', 'goat',
               'gorilla', 'horse', 'kangaroo', 'leopard', 'lion', 'panda', 'penguin', 'sheep', 'skunk', 'tiger', 'zebra']

    # Read and preprocess the image
    image = read_image(path)
    time.sleep(0.5)

    # Convert image to torch tensor
    image = torch.tensor(image, dtype=torch.float32)
    # Change dimension order to [batch_size, channels, height, width]
    image = image.permute(0, 3, 1, 2)
    image = image.to('cpu')  # Set to 'cuda' if you're using GPU

    # Extract features using the pretrained InceptionV3 model
    with torch.no_grad():
        features = pytorch_model(image)

    # Convert features to numpy array
    features = features.detach().numpy()

    # Make predictions using the trained model
    preds = model.predict(features)

    # Print individual class probabilities
    for idx, animal, prob in zip(range(0, 20), animals, preds[0]):
        print("ID: {}, Label: {} {}%".format(
            idx, animal, round(prob * 100, 2)))

    print('Final Decision:')
    time.sleep(0.5)

    # Simulate decision-making process
    for x in range(3):
        print('.'*(x+1))
        time.sleep(0.2)

    # Determine the predicted class
    class_prob = list(preds[0])
    max_prob = max(class_prob)
    pred_class = class_prob.index(max_prob)
    print("ID: {}, Label: {}".format(pred_class, animals[pred_class]))

    return load_img(path)

In [ ]:
path = 'data/test/kangaroo/5_78.jpg'

In [ ]:
test_single_image(path)